```
Authors: Ehsan Kamalinejad (EK), Emily Webber
Created: 2023-02-27
```

# RLHF

Reinforcement learning with human feedback is an interesting technique designed to aggregate human prefences at scale. In particular we train a regressive large language model built on human-desginated ranks for each prompt. Then, we use this model to serve as the `reward signal`, fine-tuning another LLM. 

Here, we present a training pipeline to finetune a generative model to create IMDb reviews with positive sentiment according to the [OpenAI RLHF paper](https://arxiv.org/abs/1909.08593)(please see section 3).

Please note, all of the training in this notebook currently happens locally, so you'll want to use an instance with enough accelerator memory. I'm running on a `ml.g4dn.xlarge`.

---
### 1. Install requirements

In [2]:
%%writefile requirements.txt
tqdm
omegaconf
dataclasses
torchtyping
datasets
transformers
torch
xformers

Writing requirements.txt


In [3]:
!pip install -r  requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.4 MB/s eta 0:00:00ta 0:00:01
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/1a/d1/3bba59606141ae808017f6fde91453882f931957f125009417b87a281067/transformers-4.34.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.5 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/e1/24/f7fe3fe82583e6891cc3fceeb390f192f6c7f1d87e5a99a949ed33c96167/torch-2.1.0-cp38-cp38-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for xformers from https://files.pythonhosted.org/packages/41/02/95d6f0abe371271941bfda567f5d062e82013f80925253fe475e3abac5b7/xformers-0.0.22-cp38-cp38-manylinux201

In [4]:
import random
import numpy as np
from tqdm.notebook import tqdm
from omegaconf import DictConfig
from dataclasses import dataclass
from typing import Optional, Tuple, Union
from typing import Iterable, Sequence, List

from torchtyping import TensorType

import transformers
from transformers import DataCollatorWithPadding
from transformers import pipeline, AutoTokenizer

from datasets import load_dataset

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR

### 2. Define model parameters

In [5]:
config = {
    'train': {
        'seed': 2023,
        'seq_length': 1024,
        'epochs': 50,
        'total_steps': 5000,
        'batch_size': 64,
        'eval_interval': 100,
        'model_device':'cuda:0',
        'ref_model_device':'cpu',
        'reward_model_device':'cpu'},
    'model': {
        'model_path': 'lvwerra/gpt2-imdb', #'edbeeching/gpt-neo-1.3B-imdb',
        'tokenizer_path': 'lvwerra/gpt2-imdb', #'edbeeching/gpt-neo-1.3B-imdb',
        'num_layers_unfrozen': 1},
    'optimizer': {
        'name': 'adamw',
        'kwargs': {'lr': 0.0001,
        'betas': [0.9, 0.95],
        'eps': 1e-08,
        'weight_decay': 1e-06}},
    'scheduler': {
        'name': 'cosine_annealing',
        'kwargs': {
            'T_max': 10000, 'eta_min': 0.0001}},
    'method': {
        'use_whitening': True,
        'prompt_size': 10,
        'num_rollouts': 128,
        'chunk_size': 128,
        'ppo_epochs': 4,
        'kl_coef': 0.05,
        'horizon': 10000,
        'gamma': 1,
        'lam': 0.95,
        'cliprange': 0.2,
        'cliprange_value': 0.2,
        'vf_coef': 1,
        'scale_reward': False,
        'ref_mean': None,
        'ref_std': None,
        'cliprange_reward': 10,
        'gen_kwargs': {
            'max_new_tokens': 60,
            'top_k': 0,
            'top_p': 1.0,
            'do_sample': True}}}

In [6]:
config = DictConfig(config)

In [7]:
random.seed(config.train.seed)
np.random.seed(config.train.seed)
torch.manual_seed(config.train.seed)
torch.cuda.manual_seed(config.train.seed)

---
### 3. Define PyTorch objects
Here, we'll show you how to implement the following reward modelling objects in PyTorch:
1. Prompt Pipeline
2. PPO RL Element
3. PPO RL Batch
4. PPO Rollout Storage, including a data loader

In [8]:
class PromptPipeline():
    def __init__(self, prompts: List[str], max_prompt_length: int, tokenizer):
        super().__init__()

        prompts = tokenizer(prompts).input_ids

        self.tokenizer = tokenizer
        self.prompts = [prompt[-max_prompt_length:] for prompt in prompts]
        self.prompts = [{"input_ids": prompt, "attention_mask": [1] * len(prompt)} for prompt in self.prompts]

    def __getitem__(self, ix: int):
        return self.prompts[ix]

    def __len__(self) -> int:
        return len(self.prompts)

    def create_loader(self, batch_size: int, shuffle=False) -> DataLoader:
        collate_fn = DataCollatorWithPadding(self.tokenizer)
        return DataLoader(self, batch_size=batch_size, collate_fn=collate_fn, shuffle=shuffle)

In [9]:
@dataclass
class PPORLElement:
    query_tensor: TensorType["query_size"]
    response_tensor: TensorType["response_size"]
    logprobs: TensorType["response_size", "vocab_size"]
    values: TensorType["response_size"]
    rewards: TensorType["response_size"]


@dataclass
class PPORLBatch:
    query_tensors: TensorType["batch_size", "query_size"]
    response_tensors: TensorType["batch_size", "response_size"]
    logprobs: TensorType["batch_size", "response_size", "vocab_size"]
    values: TensorType["batch_size", "response_size"]
    rewards: TensorType["batch_size", "response_size"]


class PPORolloutStorage():
    def __init__(self, pad_token_id):
        super().__init__()
        self.pad_token_id = pad_token_id
        self.history: Iterable[PPORLElement] = [None]

    def push(self, exps: Iterable[PPORLElement]):
        self.history += exps

    def clear_history(self):
        self.history = []

    def __getitem__(self, index: int) -> PPORLElement:
        return self.history[index]

    def __len__(self) -> int:
        return len(self.history)

    def create_loader(self, batch_size: int, shuffle: bool) -> DataLoader:
        def collate_fn(elems: Iterable[PPORLElement]):
            return PPORLBatch(
                pad_sequence(
                    [elem.query_tensor.flip(0) for elem in elems],
                    padding_value=self.pad_token_id,
                    batch_first=True,
                ).flip(1),
                pad_sequence(
                    [elem.response_tensor for elem in elems],
                    padding_value=self.pad_token_id,
                    batch_first=True,
                ),
                pad_sequence(
                    [elem.logprobs for elem in elems],
                    padding_value=0.0,
                    batch_first=True,
                ),
                pad_sequence(
                    [elem.values for elem in elems],
                    padding_value=0.0,
                    batch_first=True
                ),
                pad_sequence(
                    [elem.rewards for elem in elems],
                    padding_value=0.0,
                    batch_first=True,
                ),
            )

        return DataLoader(self, batch_size, shuffle=shuffle, collate_fn=collate_fn)

---
### 4. Define more PyTorch functions
Next, we'll implement more capabilities in PyTorch. This includes:
1. A whiten utility
2. A GAE utility
3. The loss to update the PPO LLM you want to fine-tune
4. An Actor class to take steps and experience rewards
5. An Agent class to manipulate the Actor

In [10]:
def whiten(x):
    var, mean = torch.var_mean(x)
    return (x - mean) * torch.rsqrt(var + 1e-8)


def gae(
    values,
    rewards,
):
    advantages = torch.zeros_like(rewards, device=rewards.device)
    last_advantage = 0
    last_value = 0
    
    with torch.no_grad():
        for t in reversed(range(rewards.shape[1])):
            delta = rewards[:, t] + config.method.gamma * last_value - values[:, t]
            last_advantage = delta + config.method.gamma * config.method.lam * last_advantage
            advantages[:, t] = last_advantage
            last_value = values[:, t]

        returns = advantages + values
    
    if config.method.use_whitening:
        advantages = whiten(advantages)
    
    return advantages, returns

In [11]:
def ppo_loss(
    logprobs,     
    values,       
    old_logprobs, 
    old_values,   
    advantages,   
    returns,      
    mask,         
):

    values_clipped = torch.clamp(
        values,
        old_values - config.method.cliprange_value,
        old_values + config.method.cliprange_value,
    )
    
    n = mask.sum()
    
    vf_loss1 = (values - returns) ** 2
    vf_loss2 = (values_clipped - returns) ** 2
    vf_loss = 0.5 * torch.sum(torch.max(vf_loss1, vf_loss2) * mask) / n

    log_ratio = (logprobs - old_logprobs) * mask
    ratio = torch.exp(log_ratio)
    pg_loss1 = -advantages * ratio
    pg_loss2 = -advantages * torch.clamp(ratio, 1.0 - config.method.cliprange, 1.0 + config.method.cliprange)
    pg_loss = torch.sum(torch.max(pg_loss1, pg_loss2) * mask) / n
    pg_clipfrac = torch.sum((pg_loss2 > pg_loss1).float() * mask) / n

    loss = pg_loss + config.method.vf_coef * vf_loss
    
    return loss

In [12]:
def loss_fn(batch):
    model_device = next(model.parameters()).device
    query_tensors = batch.query_tensors.to(model_device)
    response_tensors = batch.response_tensors.to(model_device)
    old_logprobs = batch.logprobs.to(model_device)
    old_values = batch.values.to(model_device)
    old_rewards = batch.rewards.to(model_device)
    
    response_length = old_rewards.shape[1]

    advantages, returns = gae(old_values, old_rewards)

    tokens, attention_mask, position_ids = get_model_inputs(query_tensors, response_tensors, tokenizer.pad_token_id)

    logits, values_pred = model(tokens,
                                attention_mask=attention_mask,
                                position_ids=position_ids)
    values_pred = values_pred[:, :-1]
    logprobs = logprobs_from_logits(logits[:, :-1, :], tokens[:, 1:])
    attention_mask = attention_mask[:, :-1]

    start = query_tensors.shape[1] - 1
    end = start + response_length
    logprobs, values_pred, mask = (
        logprobs[:, start:end],
        values_pred[:, start:end],
        attention_mask[:, start:end],
    )

    loss = ppo_loss(
        logprobs=logprobs,
        values=values_pred,
        old_logprobs=old_logprobs,
        old_values=old_values,
        advantages=advantages,
        returns=returns,
        mask=mask,
    )

    return loss, old_rewards[:,-1].mean().item()

In [13]:
class Actor():

    def __init__(
            self,
            prompt_pipeline,
            tokenizer,
            chunk_size = 128):
        
        self.prompt_pipeline = prompt_pipeline
        self.chunk_size = chunk_size

        self.prompt_pipeline_loader = self.prompt_pipeline.create_loader(self.chunk_size, shuffle=True)
        self.prompt_pipeline_iterator = iter(self.prompt_pipeline_loader)

        self.ref_model = Agent(config.model.model_path)
        self.ref_model_device = config.train.ref_model_device
        self.ref_model = self.ref_model.to(self.ref_model_device)
        
        self.tokenizer = tokenizer        
    

    def make_experience(self, model, num_rollouts = 128):
        model_device = next(model.parameters()).device
        
        ppo_rl_elements = []
        while len(ppo_rl_elements) < num_rollouts:
            try:
                batch = next(self.prompt_pipeline_iterator)
            except StopIteration:
                self.pipeline_iterator = iter(self.prompt_pipeline_loader)
                batch = next(self.prompt_pipeline_iterator)
            
            trajectories = generate(model, self.tokenizer, **batch.to(model_device))

            query_tensors = batch.input_ids
            response_tensors = trajectories[:, query_tensors.shape[1] :]

            all_tokens, attention_mask, position_ids = get_model_inputs(
                query_tensors.to(response_tensors.device), response_tensors, self.tokenizer.pad_token_id)
            with torch.no_grad():
                logits, values = model(
                    all_tokens, 
                    attention_mask=attention_mask, 
                    position_ids=position_ids)
                ref_logits, _ = self.ref_model(
                    all_tokens.to(self.ref_model_device),
                    attention_mask=attention_mask.to(self.ref_model_device),
                    position_ids=position_ids.to(self.ref_model_device))
            
            all_tokens = all_tokens.cpu()
            logits = logits.cpu()
            ref_logits = ref_logits.cpu()

            logprobs = logprobs_from_logits(logits[:, :-1, :], all_tokens[:, 1:])
            ref_logprobs = logprobs_from_logits(ref_logits[:, :-1, :], all_tokens[:, 1:])
            
            n = trajectories.shape[0]
            values = values.cpu()[:, :-1]
            query_tensors = query_tensors.cpu()
            response_tensors = response_tensors.cpu()
            
            start = query_tensors.shape[1] - 1
            ends = start + attention_mask[:, start:].sum(1)
            all_values = [values[i, start : ends[i]] for i in range(n)]
            all_logprobs = [logprobs[i, start : ends[i]] for i in range(n)]
            
            texts = self.tokenizer.batch_decode(trajectories, skip_special_tokens=True)
            scores = torch.tensor(reward_fn(texts), device='cpu', dtype=torch.float)

            rewards = -config.method.kl_coef * (logprobs - ref_logprobs)
            all_rewards = [None] * n
            for i in range(n):
                rs = rewards[i][start : ends[i]]
                rs[-1] = scores[i]
                all_rewards[i] = rs
            
            new_ppo_rl_elements = [
                PPORLElement(
                    query_tensor=query_tensors[i],
                    response_tensor=response_tensors[i],
                    logprobs=all_logprobs[i],
                    values=all_values[i],
                    rewards=all_rewards[i],
                )
                for i in range(n)
            ]

            ppo_rl_elements += new_ppo_rl_elements

        return ppo_rl_elements, scores.mean().item()

In [14]:
def generate(model, tokenizer, input_ids, attention_mask=None, **kwargs):
    
    generate_kwargs = dict(
        config.method.gen_kwargs,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id)

    kwargs = dict(generate_kwargs, **kwargs)

    with torch.no_grad():
        generated_results = model.generate(input_ids=input_ids, attention_mask=attention_mask, **kwargs)

    return generated_results


def get_model_inputs(query_tensors, response_tensors, pad_token_id):
    tokens = torch.cat((query_tensors, response_tensors), dim=1)[:, -config.train.seq_length :]
    attention_mask = (tokens.not_equal(pad_token_id).long().to(tokens.device))
    position_ids = attention_mask.cumsum(-1) - 1
    position_ids.masked_fill_(attention_mask.eq(0), 0)
    return tokens, attention_mask, position_ids


def logprobs_from_logits(logits, labels):
    logprobs = F.log_softmax(logits, dim=-1)
    logprobs_labels = torch.gather(logprobs, dim=-1, index=labels.unsqueeze(-1))
    return logprobs_labels.squeeze(-1)


def freeze_bottom_causal_layers(model: nn.Module, num_layers_unfrozen: int = 0):
    hidden_layers = model.transformer.h
    if num_layers_unfrozen == 0:
        hidden_layers_to_freeze = list(hidden_layers)
    elif num_layers_unfrozen > 0:
        hidden_layers_to_freeze = list(hidden_layers)[:-num_layers_unfrozen]
    else:
        hidden_layers_to_freeze = []
    for layer in hidden_layers_to_freeze:
        layer.requires_grad_(False)

        
class Agent(nn.Module):
    def __init__(self, model_path, num_layers_unfrozen=0):
        super().__init__()

        self.base_model = transformers.AutoModelForCausalLM.from_pretrained(model_path, cache_dir="./models")

        self.logit_head = self.base_model.get_output_embeddings()
        
        n_embd = self.base_model.lm_head.in_features
        self.value_head = nn.Sequential(
            nn.Linear(n_embd, n_embd*2),
            nn.ReLU(),
            nn.Linear(n_embd*2, 1))
        
        freeze_bottom_causal_layers(self.base_model, num_layers_unfrozen)
        
    
    def generate(self, input_ids, **x):
        return self.base_model.generate(input_ids, **x)

    def forward(self, input_ids, attention_mask, position_ids):

        transformer_outputs = self.base_model.transformer(input_ids=input_ids,
                                                          attention_mask=attention_mask,
                                                          position_ids=position_ids)
    
        last_hidden_state = transformer_outputs.last_hidden_state
        lm_logits = self.logit_head(last_hidden_state)
        value = self.value_head(last_hidden_state).squeeze(-1)
        
        return lm_logits, value

---
### 5. Define the pipeline, download model and data artifacts

In [15]:
sentiment_fn = pipeline(
    model = "lvwerra/distilbert-imdb",
    top_k=2,
    batch_size=config.method.num_rollouts,
    device=config.train.reward_model_device,
)

In [16]:
def get_positive_score(scores):
    return dict(map(lambda x: tuple(x.values()), scores))["POSITIVE"]

def reward_fn(samples: List[str]) -> List[float]:
    sentiments = list(map(get_positive_score, sentiment_fn(samples)))
    return sentiments

In [17]:
imdb = load_dataset("imdb", split="train+test")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [18]:
prompts = [" ".join(review.split()[:config.method.prompt_size]) for review in imdb["text"]]

In [19]:
tokenizer = AutoTokenizer.from_pretrained(config.model.tokenizer_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_ida = tokenizer.eos_token_id
tokenizer.padding_side = "left"
pad_token_id = 50256

max_prompt_length = (config.train.seq_length - config.method.gen_kwargs["max_new_tokens"])
test_prompt_pipeline = PromptPipeline(prompts, max_prompt_length, tokenizer)

In [20]:
model = Agent(config.model.model_path, config.model.num_layers_unfrozen).to(config.train.model_device)

In [21]:
input_ids = tokenizer.batch_encode_plus(
    ["my feeling about the movie", "this is", "I can tell with certainty"],
    return_tensors='pt',
    padding=True)['input_ids']
input_ids

tensor([[ 1820,  4203,   546,   262,  3807],
        [50256, 50256, 50256,  5661,   318],
        [   40,   460,  1560,   351, 18979]])

In [22]:
model_device = next(model.parameters()).device
output_ids = generate(
    model, tokenizer, 
    input_ids.to(model_device), 
    max_new_tokens=config.method.gen_kwargs["max_new_tokens"]
)

In [23]:
generated_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
generated_text

['my feeling about the movie, as well as the style and tone of the famous "Amadeus" music films.',
 'this is just awful! it makes you feel very lonely. it is not until next day Italy that you can see epitomise how bad this film was. i was trying to find something im going down the road to television, whilst watching this I felt humiliated, i totally felt embarrassed i have never felt',
 'I can tell with certainty that Pulp, Chicago Voice and I like Victor Oppel that he is correct and his Molly Hallrem was one amazing film --a Mustivar of Terror, and a tremendous piece of exceptional film.On well getting past the Great Affair in Vincent Van Buren, Democracy in Love is the']

In [24]:
reward_fn(generated_text)

[0.988457977771759, 0.006799057591706514, 0.9953449368476868]

--- 
### 6. Define the overall training loop and run

In [25]:
prompt_pipeline = PromptPipeline(prompts, config.train.seq_length, tokenizer)

actor = Actor(prompt_pipeline, tokenizer, chunk_size=config.method.chunk_size)

store = PPORolloutStorage(tokenizer.pad_token_id)

opt = torch.optim.Adam(model.parameters(), **config.optimizer.kwargs)
scheduler = CosineAnnealingLR(opt, **config.scheduler.kwargs)

n_updates_per_batch = config.method.ppo_epochs
total_steps = 400 # TODO: fix this

tbar = tqdm(initial=0, total=total_steps)

for _ in range(config.train.epochs):
    
    store.clear_history()
    rollouts, score = actor.make_experience(model, config.method.num_rollouts)
    store.push(rollouts)
    train_dataloader = store.create_loader(config.train.batch_size, shuffle=True)
    
    for batch in train_dataloader:
        for _ in range(n_updates_per_batch):

            loss, reward = loss_fn(batch)
            loss.backward()
            opt.step()
            opt.zero_grad()
            scheduler.step()
            tbar.update()
    
    tbar.set_description(f"| score: {score:.3f} |")

  0%|          | 0/400 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


---
### 7. Results: Use the updated LLM to generate new text and evaluate with the reward model

In [26]:
input_ids = tokenizer.batch_encode_plus(
    ["my feeling about the movie", "this is", "I can tell with certainty"],
    return_tensors='pt',
    padding=True)['input_ids']
input_ids

tensor([[ 1820,  4203,   546,   262,  3807],
        [50256, 50256, 50256,  5661,   318],
        [   40,   460,  1560,   351, 18979]])

In [27]:
model_device = next(model.parameters()).device
output_ids = generate(
    model,       
    tokenizer,
    input_ids.to(model_device),
#     min_length=20,
#     max_new_tokens=100,
#     do_sample=True,
#     top_p=0.92, 
#     top_k=0
)
generated_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
rewards = reward_fn(generated_text)
print(generated_text[0].replace('\n', ' ') + '\n', rewards[0])
print(generated_text[1].replace('\n', ' ') + '\n', rewards[1])
print(generated_text[2].replace('\n', ' ') + '\n', rewards[2])
print('all rewards mean:',np.mean(rewards))

my feeling about the movie-binding machine was palpable throughout.<br /><br />I am glad I spent more time watching this again, because hopefully I'll watch as many films as I are good at.
 0.9671000838279724
this is a wonderful film and a gem.
 0.9961223006248474
I can tell with certainty that this movie has a great deal in common with the Harry Potter trilogy or the Harry Potter films: it's a great treat for kids and adults alike.
 0.9945493340492249
all rewards mean: 0.9859239061673483
